In [ ]:
''' Algoritmo realizado para a disciplina de Tecnologia e aplicação de informações quânticas UNILA 2024 GSP

f(x0, x1, x2) = 2.x0 -4.x0.x1 + 3.x1.x2
xi = (1-zi)/2 ==> troca de variavel de QUBO para Ising

 g(z0, z1, z2) = 3/4 + z1/4 -3.z2/4 - z0.z1 + 3.z1.z2/4

 1 1 0 QUBO ==> -1 -1 1  Ising '''

In [39]:
import pennylane as qml


In [40]:
variaveis = 3

In [25]:
dev = qml.device("default.qubit", wires = variaveis, shots = 1000) # chama a máquina 

In [26]:
#função para realizar a rotação em x para todos os segmentos
def phi(w):
    qml.RX(w[0], wires = 0)
    qml.RX(w[1], wires = 1)
    qml.RX(w[2], wires = 2)

In [27]:
# Funcoes circ ==> retornam o valor esperado de um termo do hamiltoniano (cada aplicacao das matrizes de pauli geram um novo termo)

In [28]:
@qml.qnode(dev)
def circ1(w):
    phi(w)
    return qml.expval(qml.Identity(wires = 0) @ qml.Identity(wires = 1) @ qml.Identity(wires = 2))

In [29]:
@qml.qnode(dev)
def circ2(w):
    phi(w)
    return qml.expval(qml.Identity(wires = 0) @ qml.PauliZ(wires = 1) @ qml.Identity(wires = 2))

In [30]:
@qml.qnode(dev)
def circ3(w):
    phi(w)
    return qml.expval(qml.Identity(wires = 0) @ qml.Identity(wires = 1) @ qml.PauliZ(wires = 2))
    

In [31]:
@qml.qnode(dev)
def circ4(w):
    phi(w)
    return qml.expval(qml.PauliZ(wires = 0) @ qml.PauliZ(wires = 1) @ qml.Identity(wires = 2))


In [32]:
@qml.qnode(dev)
def circ5(w):
    phi(w)
    return qml.expval(qml.Identity(wires = 0) @ qml.PauliZ(wires = 1) @ qml.PauliZ(wires = 2))


In [33]:
def valor(w):
    return 3*circ1(w)/4 + circ2(w)/4 - 3*circ3(w)/4 - circ4(w) + 3*circ5(w)/4
# devolve o valor da funcao aplicada com os termos em Ising ==> g(z0,z1,z2)

In [34]:
w = qml.numpy.random.rand(3) * 2*3.14 #cria um valor aleatorio

In [35]:
gradient_fn_w = qml.grad(valor, argnum=0) #funcao de minimizacao


In [37]:
#for para aplicar a minimizacao varias vezes
lr = 0.005
for rep in range(501):
    w = w - lr*gradient_fn_w(w)
    if rep % 50 == 0:
        print("rep", rep, "perda", valor(w))

@qml.qnode(qml.device("default.qubit", wires = variaveis, shots = 1))
# funcao para dar a solucao final de otimizacao
def solucao(w):   
    phi(w)
    return [qml.sample(qml.PauliZ(i)) for i in range(3)]

solucao(w)

rep 0 perda -1.9669999999999999
rep 50 perda -1.9805
rep 100 perda -1.992
rep 150 perda -1.994
rep 200 perda -2.0
rep 250 perda -2.0
rep 300 perda -2.0
rep 350 perda -2.0
rep 400 perda -2.0
rep 450 perda -2.0
rep 500 perda -2.0


[array(-1, dtype=int64), array(-1, dtype=int64), array(1, dtype=int64)]